In [ ]:
import pandas as pd
import recordlinkage

Importazione dataset.

In [ ]:
stazioni_vt = pd.read_csv('stazioni_vt.csv')
stazioni_vt.drop_duplicates(inplace=True)
stazioni_vt['nome_stazione'] = stazioni_vt['nome_stazione'].str.lower()
stazioni_vt['nome_stazione'] = stazioni_vt['nome_stazione'].str.replace(' ', '-', regex=False)
stazioni_vt['nome_stazione'] = stazioni_vt['nome_stazione'].str.replace('.', '-', regex=False)
stazioni_vt['nome_stazione'] = stazioni_vt['nome_stazione'].str.replace('s-', 'san-', regex=False)
stazioni_vt['nome_stazione'] = stazioni_vt['nome_stazione'].str.replace('sant-', 'san-', regex=False)
stazioni_vt['nome_stazione'] = stazioni_vt['nome_stazione'].str.replace('c-le', 'centrale', regex=False)
stazioni_vt['nome_stazione'] = stazioni_vt['nome_stazione'].str.replace('[a-z]`', '-', regex=True)
stazioni_vt['nome_stazione'] = stazioni_vt['nome_stazione'].str.replace('--', '-', regex=False)
stazioni_vt.rename(columns={'nome_stazione':'nome_stazione_vt'}, inplace=True)

In [ ]:
stazioni_rfi = pd.read_json('/home/luca/GoogleDriveUni/Universita/Data Management/Progetto Data Man/stazioni_binari.json', orient = 'index')
stazioni_rfi.reset_index(inplace=True)
stazioni_vt.drop_duplicates(inplace=True)
stazioni_rfi.drop(0, axis=1, inplace=True)
stazioni_rfi['index'] = stazioni_rfi['index'].str.replace(' ', '-', regex=False)
stazioni_rfi['index'] = stazioni_rfi['index'].str.replace('.', '-', regex=False)
stazioni_rfi['index'] = stazioni_rfi['index'].str.replace('s-', 'san-', regex=False)
stazioni_rfi['index'] = stazioni_rfi['index'].str.replace('sant-', 'san-', regex=False)
stazioni_rfi['index'] = stazioni_rfi['index'].str.replace('c-le', 'centrale', regex=False)
stazioni_rfi['index'] = stazioni_rfi['index'].str.replace('--', '-', regex=False)
stazioni_rfi.rename(columns={'index':'nome_stazione_rfi'}, inplace=True)

Record linkage.

In [ ]:
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(stazioni_vt, stazioni_rfi)

In [ ]:
compare_cl = recordlinkage.Compare()
compare_cl.string('nome_stazione_vt', 'nome_stazione_rfi', method="jarowinkler", label='nome_stazioni')
features = compare_cl.compute(pairs, stazioni_vt, stazioni_rfi)

In [ ]:
features_match = features[features['nome_stazioni'] == 1]
features_match.reset_index(inplace=True)

In [ ]:
stazioni_vt_nm = stazioni_vt.drop(features_match['level_0'])
stazioni_rfi_nm = stazioni_rfi.drop(features_match['level_1'])

In [ ]:
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(stazioni_vt_nm, stazioni_rfi_nm)

In [ ]:
compare_cl = recordlinkage.Compare()
compare_cl.string('nome_stazione_vt', 'nome_stazione_rfi', method="jarowinkler", label='nome_stazioni')
features_2 = compare_cl.compute(pairs, stazioni_vt_nm, stazioni_rfi_nm)

In [ ]:
features_match_2 = features_2[features_2['nome_stazioni'] > 0.9]
features_match_2.reset_index(inplace=True)
features_match_2 = features_match_2.loc[features_match_2.groupby('level_0')['nome_stazioni'].idxmax()]
final = pd.concat([features_match, features_match_2])
final.rename(columns={'level_0':'id_vt', 'level_1':'id_rfi','nome_stazioni':'somigl_nomi_stazioni'}, inplace=True)
final.to_csv('matching stazioni_vt-stazioni_rfi.csv')

In [ ]:
stazioni_mancanti = stazioni_vt_nm.drop(features_match_2['level_0'])
stazioni_mancanti.to_csv('stazioni mancanti.csv', index=False, header=False)